In [1]:
import pandas as pd

# 파일 경로
file_path = "data/yelp_academic_dataset_review.json"

# chunk 단위로 전체 데이터 읽고 합치기
chunk_size = 100000
chunks = pd.read_json(file_path, lines=True, chunksize=chunk_size)

# 모든 chunk를 리스트에 담아서 연결
df_r = pd.concat(chunk for chunk in chunks)

In [4]:
df = df_r.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 480.0+ MB


In [6]:
import pandas as pd
import json
import re

def clean_review(text):
    text = text.strip().lower()                    # 소문자화 + 양쪽 공백 제거
    text = re.sub(r"http\S+", "", text)            # URL 제거
    text = re.sub(r"[^a-z0-9\s.,!?']", " ", text)  # 알파벳/숫자/기초 문장부호 외 제거
    text = re.sub(r"\s+", " ", text)               # 공백 정리
    return text

# funny와 cool 컬럼 제거
df = df.drop(columns=["funny", "cool"])

df["cleaned_text"] = df["text"].astype(str).apply(clean_review)

In [8]:
import time
from tqdm import tqdm
from transformers import DebertaV2Tokenizer

# 1. 토크나이저 로드
tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-base")

# 2. 토큰 길이 계산 (진행률 + 시간 측정)
start = time.time()

df["token_length"] = [
    len(tokenizer.tokenize(text)) for text in tqdm(df["cleaned_text"], desc="🔍 토큰 길이 계산", total=len(df))
]

# 3. 필터링: 토큰 길이 1~128 사이만 유지
before = len(df)
df = df[(df["token_length"] > 0) & (df["token_length"] <= 128)]
after = len(df)

end = time.time()
elapsed = end - start
minutes = int(elapsed // 60)
seconds = int(elapsed % 60)

print(f"✅ 토큰 필터링 완료: {before - after}개 리뷰 제거됨 (총 {before} → {after})")
print(f"⏱ 소요 시간: {minutes}분 {seconds}초")

🔍 토큰 길이 계산: 100%|██████████| 6990280/6990280 [19:06<00:00, 6096.01it/s]


✅ 토큰 필터링 완료: 2316286개 리뷰 제거됨 (총 6990280 → 4673994)
⏱ 소요 시간: 19분 37초


In [10]:
import json

# 저장 경로
output_path = "data/review_0509.json"

# 1. datetime → ISO 문자열로 변환
df["date"] = df["date"].astype(str)  # 또는 df["date"].dt.strftime("%Y-%m-%d")

with open(output_path, "w", encoding="utf-8") as f:
    for row in df.to_dict(orient="records"):
        json.dump(row, f, ensure_ascii=False)
        f.write("\n")

print(f"📁 전체 컬럼 포함 JSONL 저장 완료: {output_path}")
print(f"✅ 저장된 리뷰 수: {len(df)}")


📁 전체 컬럼 포함 JSONL 저장 완료: data/review_0509.json
✅ 저장된 리뷰 수: 4673994


# 근거

① 데이터 Chunk 단위 처리 근거
대규모 데이터셋을 처리할 때 메모리 효율성을 높이기 위한 Chunk 단위 처리 방법입니다.

참고문헌:
McKinney, W. (2011). pandas: a foundational Python library for data analysis and statistics. Python for High Performance and Scientific Computing, 14(9), 1-9.

🔗 출처

이 문헌은 pandas 라이브러리를 사용한 효율적 데이터 처리 기법으로 chunk 기반 데이터 로딩의 중요성을 설명합니다.

② 텍스트 데이터 전처리 근거
머신러닝 모델(특히 NLP 모델)을 훈련하기 전 텍스트 데이터를 정제하는 과정의 중요성을 설명하는 문헌입니다.

참고문헌:
Haddi, E., Liu, X., & Shi, Y. (2013). The role of text pre-processing in sentiment analysis. Procedia Computer Science, 17, 26-32.

🔗 출처 https://www.sciencedirect.com/science/article/pii/S1877050913001385

이 논문은 텍스트 전처리 과정이 감성 분석 모델의 정확도와 성능에 매우 큰 영향을 미친다고 강조합니다.

③ 토큰화 및 길이 기반 필터링 근거
모델 훈련 시 최대 길이를 기준으로 입력 데이터를 필터링하여 효율성을 높이는 방법입니다.

참고문헌:
Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. Proceedings of NAACL 2019.

🔗 출처 https://aclanthology.org/N19-1423/

이 논문은 Transformer 기반 모델에서 입력 시퀀스의 최대 길이를 설정하고 초과하는 데이터를 자르거나 제거하는 것이 효율적임을 언급하며, 성능과 훈련 속도 개선에 영향을 준다고 보고합니다.